<a href="https://colab.research.google.com/github/160445092/colab_test2/blob/main/hum_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [7]:

#バターワースフィルタ（バンドストップ）
def bandstop(x, samplerate, fp, fs, gpass, gstop):
    fn = samplerate / 2
    print(fp)                           #ナイキスト周波数
    wp = np.array(fp / 500)                                  #ナイキスト周波数で通過域端周波数を正規化
    ws = np.array(fs / 500)                                #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "bandstop")       #フィルタ伝達関数の分子と分母を計算
    y = signal.filtfilt(b, a, x)                  #信号に対してフィルタをかける
    return y         

##関数lowpass定義

In [8]:
import numpy as np
from scipy import signal
import math

#バターワースフィルタ（ローパス）
def lowpass(x, samplerate, fp, fs, gpass, gstop):
    ### 注意　fp<fs じゃないと使えないかも ###

    ### データ補間 2の累乗に合わせる ###
    yososu=len(x)
    print(yososu) #確認
    njo=math.ceil((math.log2(yososu))) #2の何乗？
    hoju=2**njo-1-yososu
    if hoju != 0:
        dainyu=np.full(hoju,x[-1])
        x=np.append(x,dainyu)
    ### データ補間　ここまで ###

    fn = samplerate / 2                           #ナイキスト周波数
    wp = fp / fn                                  #ナイキスト周波数で通過域端周波数を正規化
    ws = fs / fn                                  #ナイキスト周波数で阻止域端周波数を正規化
    N, Wn = signal.buttord(wp, ws, gpass, gstop)  #オーダーとバターワースの正規化周波数を計算
    b, a = signal.butter(N, Wn, "low")            #フィルタ伝達関数の分子と分母を計算
    y = signal.filtfilt(b, a, x)                  #信号に対してフィルタをかける
    y = y[:yososu]                              #はじめに受け取った要素数を返す
    return y 

##関数FFT定義


In [9]:
import numpy as np
import math

def FFT(x, dt):

    ### データ補間 2の累乗に合わせる ###
    yososu=len(x)
    print(yososu) #確認
    njo=math.ceil((math.log2(yososu))) #2の何乗？
    hoju=2**njo-1-yososu
    if hoju != 0:
        dainyu=np.full(hoju,x[-1])
        x=np.append(x,dainyu)
    ### データ補間　ここまで ###

    ### FFT ###

    ## 窓関数
    window_n = np.hanning(len(x))
    # x=x*window_n #周期的な信号のときに使ったほうがよい？
    #print(window_n)

    ## 処理
    F = np.fft.fft(x) # 変換結果
    freq = np.fft.fftfreq(len(x), d=dt) # 周波数
    Amp = np.abs(F/(len(x)/2))   # 振幅

    freq=freq[1:int(len(x)/2)]
    Amp=Amp[1:int(len(x)/2)]

    return freq, Amp 

##全文

In [10]:

""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
"""
#定義ファイル読み込み
import sys
sys.path.append('gdrive/My Drive/Colab Notebooks/EMG_estimation/EMG_hum_filter')
import untitled1_py
"""
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

import glob
import numpy as np
import csv
import matplotlib.pyplot as plt

""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

#処理設定
#処理するデータのあるフォルダの絶対パスを""内に入力
folder_path = "gdrive/My Drive/Colab Notebooks/EMG_estimation/EMG_hum_filter"
#サンプリングレート(Hz)
samplingrate = 1000

""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

### 同じフォルダ中のcsvの読み込み ###
list=glob.glob("{}/*.csv".format(folder_path))
#print(list)
if list == []:
  print("リストは空です")
  exit()


for x in list:

    ### ファイル名によって処理を変えたい場合こちらで分岐 ###
    '''
    if 'no1.csv' in x:
        print('no1') ### ファイル名にno1がついたときの処理
    elif '_no2.csv' in x:
        print('no2') ### ファイル名にno2がついたときの処理
    elif '*_no3.csv' in x:
        print('no3') ### ファイル名にno3がついたときの処理
    else:
        print("error")
    '''

    data_tot=[]  #配列を用意
    i=0
    ### CSVの読み込み　自分は行ごとに読み込むのが好み。 ###
    ### 大体のデータロガーは前半に色々な情報書いているため ###
    with open(x,mode="r",encoding="utf-8") as fp:
        csv_reader=csv.reader(fp,delimiter=",") #,区切りでないファイルのときdelimiter
        for row in csv_reader:
            if i>=1:                    #y行目以降を追加。日本語読み込むとエラーになるため
                data_tot.append(row)    #headerは無視
            i=i+1
            print(i)
    print(data_tot) #確認
    data_tot = np.array(data_tot, np.float32) #要素を型変換。csv_readerは文字列
    #print(data_tot) #確認
    """
    ### 大体のデータの1列目は時間 ###
    dt=data_tot[1,0]-data_tot[0,0]
    samplerate=1/dt
    """
    samplerate = float(samplingrate)
    dt = 1/samplerate

    shori_r=4 #処理したい列
    data=data_tot[:,shori_r]

    ### lowpass ###
    print(data) #確認
    """
    fp = 45      #通過域端周波数[Hz]
    fs = 75      #阻止域端周波数[Hz] fp<fs
    gpass = 3       #通過域端最大損失[dB]
    gstop = 40      #阻止域端最小損失[dB]
    """
    ###バンドストップフィルタの設定###
    fp = np.array([45, 75])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([55, 65])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data, samplerate, fp, fs, gpass, gstop)

    ###バンドストップフィルタの設定###
    fp = np.array([105, 135])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([115, 125])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)

    ###バンドストップフィルタの設定###
    fp = np.array([165, 195])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([175, 185])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)

    ###バンドストップフィルタの設定###
    fp = np.array([225, 255])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([235, 245])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)

    ###バンドストップフィルタの設定###
    fp = np.array([285, 315])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([295, 305])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)

    ###バンドストップフィルタの設定###
    fp = np.array([345, 375])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([355, 365])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)    

    ###バンドストップフィルタの設定###
    fp = np.array([405, 435])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([415, 425])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)

    ###バンドストップフィルタの設定###
    fp = np.array([465, 495])         #通過域端周波数[Hz]※ベクトル
    fs = np.array([475, 485])         #阻止域端周波数[Hz]※ベクトル
    gpass = 3                       #通過域端最大損失[dB]
    gstop = 40                      #阻止域端最小損失[dB]
    data_filt = bandstop(data_filt, samplerate, fp, fs, gpass, gstop)

    ### FFT ###
    fft, amp = FFT(data, dt)         #処理前データのFFT
    fft2, amp2 = FFT(data_filt, dt)  #処理後データのFFT

    ### グラフにしてpngで保存 ###
    ### 保存する紙の広さ ###
    fig=plt.figure(figsize=(18,12)) # 縦横比　(1:1.4) がきれいかも　サイズは(横,縦)

    ax1=fig.add_subplot(2,3,1)   #縦,横,何番目
    ax1.grid()
    ax1.plot(data_tot[:,0],data_tot[:,shori_r],\
             color="red",linestyle="-",\
             label="non-noize") ### ここで横軸の長さをそろえておくと報告書に載せやすい
    ax1.legend() #凡例表示
    ax1.set_xlim([0,15000]) #軸調整 subplot!!
    ax1.set_ylim([0.5,5.0]) #軸調整 subplot!!

    ax2=fig.add_subplot(2,3,2)
    ax3=ax2.twinx()  #2軸使ってみる
    ax2.grid()
    """
    ax2.plot(data_tot[:,0],data_tot[:,shori_r],\
             color="red",linestyle="-",\
             label="shori_taisho")
    """
    ax3.plot(data_tot[:,0],data_filt[:],\
             color="blue",linestyle="-",\
             label="shori")
    ax2.legend(loc="best") #凡例　左上に
    ax3.legend(loc="best") #凡例　右上に
    
    ax2.set_xlim([0,15000]) #軸調整 subplot!!
    ax2.set_ylim([1.5,3.0]) #軸調整 subplot!!
    
    ax3.set_ylim([1.5,3.0]) #軸調整 subplot!!

    ax4=fig.add_subplot(2,3,4)   #縦,横,何番目
    ax4.grid()
    ax4.plot(fft,amp,\
             color="red",linestyle="-",\
             label="fft") ### ここで横軸の長さをそろえておくと報告書に載せやすい

    ax4.legend(loc="upper left") #凡例 左上

    ax5=ax4.twinx() 
    ax5.plot(fft2,amp2,\
             color="blue",linestyle="-",\
             label="fft shori")
    ax5.legend(loc="best") #凡例　右上

    ### 一番右は余白にしてみる ###
    fig.savefig(x[:-3]+"png")
    ### グラフを閉じる。これをしないとメモリーが消費される ###
    plt.close()
    plt.clf()

ストリーミング出力は最後の 5000 行に切り捨てられました。
10978
10979
10980
10981
10982
10983
10984
10985
10986
10987
10988
10989
10990
10991
10992
10993
10994
10995
10996
10997
10998
10999
11000
11001
11002
11003
11004
11005
11006
11007
11008
11009
11010
11011
11012
11013
11014
11015
11016
11017
11018
11019
11020
11021
11022
11023
11024
11025
11026
11027
11028
11029
11030
11031
11032
11033
11034
11035
11036
11037
11038
11039
11040
11041
11042
11043
11044
11045
11046
11047
11048
11049
11050
11051
11052
11053
11054
11055
11056
11057
11058
11059
11060
11061
11062
11063
11064
11065
11066
11067
11068
11069
11070
11071
11072
11073
11074
11075
11076
11077
11078
11079
11080
11081
11082
11083
11084
11085
11086
11087
11088
11089
11090
11091
11092
11093
11094
11095
11096
11097
11098
11099
11100
11101
11102
11103
11104
11105
11106
11107
11108
11109
11110
11111
11112
11113
11114
11115
11116
11117
11118
11119
11120
11121
11122
11123
11124
11125
11126
11127
11128
11129
11130
11131
11132
11133
11134
11135
11136
11137
11138
11

No handles with labels found to put in legend.


<Figure size 432x288 with 0 Axes>